In [1]:
import pandas as pd
import processing
from graphs import *

DATA_PATH = "./data/courses.json"

courses_df = pd.read_json(DATA_PATH, orient="index").dropna()

In [2]:
outline = courses_df["outline_level_year"]
start = courses_df["delivery_start"]

courses_df["year"] = pd.to_numeric(outline.apply(processing.year))
courses_df["start_semester"] = pd.to_numeric(start.apply(processing.semester))

courses_df["start_score"] = courses_df["year"] + courses_df["start_semester"] * 0.1
courses_df = courses_df.dropna()

prereqs = courses_df["entry_pre_req"].apply(processing.prereq)

In [3]:
G = PriorityGraph()

course_to_vertex = {}
vertex_to_course = {}

for course, _ in prereqs.items():
    i = G.addVertex(courses_df["start_score"][course])
    course_to_vertex[course] = i
    vertex_to_course[i] = course

for course, prereq in prereqs.items():
    for pre_course in prereq:
        if pre_course in course_to_vertex.keys():
            G.addEdge(course_to_vertex[pre_course], course_to_vertex[course])


In [4]:
L = G.topSort()
course_order = processing.list_order(map(lambda i : courses_df.loc[vertex_to_course[i]]["title"], L))

In [5]:
G = Graph()

course_to_vertex = {}
vertex_to_course = {}

for course, _ in prereqs.items():
    i = G.addVertex()
    course_to_vertex[course] = i
    vertex_to_course[i] = course

for course, prereq in prereqs.items():
    for pre_course in prereq:
        if pre_course in course_to_vertex.keys():
            # We want this graph to be undirected
            G.addEdge(course_to_vertex[pre_course], course_to_vertex[course])
            G.addEdge(course_to_vertex[course], course_to_vertex[pre_course])

CC = G.connComp()
CC = sorted(CC, key=lambda l : len(l), reverse=True)